In [ ]:
import unicodedata
import re
import json
import os
from requests import get
from bs4 import BeautifulSoup
import acquire
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
from acquire import parse_blog
from acquire import get_article_text
from acquire import get_codeup_blogs
from acquire import get_inshorts_articles
from acquire import prep_text

In [ ]:
# Store contents of 'spam.csv' into a variable
spam = pd.read_csv('spam.csv')

In [ ]:
# Check column names.
spam.columns

In [ ]:
# Drop unwanted columns.
spam.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace=True)

In [ ]:
# Rename columns.
spam.rename(columns={'v1':'label','v2':'text'}, inplace=True)

In [ ]:
spam

In [ ]:
# Add words to append to list of stopwords.
ADDITIONAL_STOPWORDS = ['r','u','2','ltgt']

# This function will append list of words to stopwords
def clean(text):
    # Create lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    # Create list of stopwords
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    # Convert string to ASCII character set.
    text = (unicodedata.normalize('NFKD', text)
           .encode('ascii','ignore')
           .decode('utf-8','ignore')
           .lower())
    # Remove special characters.
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [ ]:
# This function shows number of words and the ratio.
def show_counts_and_ratios(df, column):
    labels = pd.concat([spam.label.value_counts(),
                       spam.label.value_counts(normalize=True)], axis =1)
    labels.columns = ['n', 'percent']
    return labels

In [ ]:
show_counts_and_ratios(spam, 'label')

In [ ]:
# Convert dataframe into text.
ham_words = clean(' '.join(spam[spam.label == 'ham'].text))
spam_words = clean(' '.join(spam[spam.label == 'spam'].text))
all_words = clean(' '.join(spam.text))

In [ ]:
# Check top 5 words in each list.
ham_words[:5], spam_words[:5], all_words[:5]

In [ ]:
# Get value counts of each word in each list.
ham_freq = pd.Series(ham_words).value_counts()
spam_freq = pd.Series(spam_words).value_counts()
all_freq = pd.Series(all_words).value_counts()

spam_freq.head()

In [ ]:
pd.concat([all_freq, ham_freq, spam_freq], axis=1, sort=True)

In [ ]:
# Combine value counts of words into one dataframe to work on

word_counts = (pd.concat([all_freq, ham_freq, spam_freq], axis=1, sort=True)
              .set_axis(['all', 'ham', 'spam'], axis=1, inplace=False)
              .fillna(0) 
              .apply(lambda s: s.astype(int)))

word_counts

### Are there any words that uniquely identify a spam or ham message?

In [ ]:
pd.concat([word_counts[word_counts.ham == 0].sort_values(by='spam', ascending=False).head(10),
          word_counts[word_counts.spam == 0].sort_values(by='ham', ascending=False).head(10)])

### What are the most frequently occuring words?

In [ ]:
word_counts.sort_values(by='all', ascending=False)

## Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Plot the ratios of spam and ham words.
(word_counts
.assign(p_spam=word_counts.spam / word_counts['all'],
        p_ham=word_counts.ham / word_counts['all'])
.sort_values(by='all')
 [['p_spam', 'p_ham']]
 .tail(20)
 .sort_values('p_ham')
 .plot.barh(stacked=True))

plt.title('Proportion of Spam vs Ham for the 20most common words')

In [ ]:
# Get values and ratios of all words that have a value count greater than 10.
(word_counts
[(word_counts.spam > 10) & (word_counts.ham > 10)]
.assign(ratio=lambda df: df.spam / (df.ham + .01))
.sort_values(by='ratio')
.pipe(lambda df: pd.concat([df.head(), df.tail()])))

In [ ]:
from wordcloud import WordCloud

# Create sentence for wordcloud
sentence = "Mary had a little lamb, little lamb, little lamb. Its fleece was white as snow."

# Generate the image using previously stored sentence.
img = WordCloud(background_color='white').generate(sentence)

# Display image.
plt.imshow(img)
plt.axis('off')

In [ ]:
# Generate images from joining all words in the list into one 'sentence' using '.join'
all_cloud = WordCloud(background_color='white', height=1000, width=400).generate(' '.join(all_words))
ham_cloud = WordCloud(background_color='white', height=600, width=800).generate(' '.join(ham_words))
spam_cloud = WordCloud(background_color='white', height=600, width=800).generate(' '.join(spam_words))

plt.figure(figsize=(10,8))
axs = [plt.axes([0,0,.5,1]), plt.axes([.5,.5,.5,.5]), plt.axes([.5,0,.5,.5])]

axs[0].imshow(all_cloud)
axs[1].imshow(ham_cloud)
axs[2].imshow(spam_cloud)

axs[0].set_title('All Words')
axs[1].set_title('Ham')
axs[2].set_title('Spam')

for ax in axs: ax.axis('off')

## Bigrams

In [ ]:
sentence = "Mary had a little lamb"

bigrams = nltk.ngrams(sentence.split(), 2)
list(bigrams)

In [ ]:
top_20_ham_bigrams = (pd.Series(nltk.ngrams(ham_words, 2))
                     .value_counts()
                     .head(20))

top_20_ham_bigrams

In [ ]:
top_20_ham_bigrams.reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1])

In [ ]:
top_20_ham_bigrams.plot.barh(color='pink', width=.9, figsize=(10,6))

plt.title('20 Most Frequently Occuring Ham Bigrams')
plt.ylabel('Bigram')
plt.xlabel('Number of Occurances')

ticks , _ = plt.yticks()
labels = top_20_ham_bigrams.reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1])
_ = plt.yticks(ticks, labels)

In [ ]:
data = {k[0] + ' ' + k[1]: v for k, v in top_20_ham_bigrams.to_dict().items()}
img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
plt.figure(figsize=(8,4))
plt.imshow(img)
plt.axis('off')
plt.show()

## Create and explore bigrams for the spam data. Visualize them with a word cloud. How do they compare with the ham bigrams?

In [ ]:
top_20_spam_bigrams = (pd.Series(nltk.ngrams(spam_words, 2))
                      .value_counts()
                      .head(20))

top_20_spam_bigrams

In [ ]:
top_20_spam_bigrams.plot.barh(color='blue', width=.9, figsize=(10,6))

plt.title('20 Most Frequently Occuring Spam Bigrams')
plt.ylabel('Bigram')
plt.xlabel('Number of Occurances')

# Make the labels pretty
ticks, _ = plt.yticks()
labels = top_20_spam_bigrams.reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1])
_ = plt.yticks(ticks, labels)

In [ ]:
data = {k[0] + ' ' + k[1]: v for k, v in top_20_spam_bigrams.to_dict().items()}
img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
plt.figure(figsize=(8,4))
plt.imshow(img)
plt.axis('off')
plt.show()

## Create and explore with trigrams(i.e. an n-gram with an n of 3) for both the spam and ham data.

In [ ]:
top_20_ham_trigrams = (pd.Series(nltk.ngrams(ham_words, 3))
                     .value_counts()
                     .head(20))

top_20_ham_trigrams

In [ ]:
top_20_ham_trigrams.plot.barh(color='pink', width=.9, figsize=(10,6))

plt.title('20 Most Frequently Occuring Ham Trigrams')
plt.ylabel('Trigram')
plt.xlabel('Number of Occurances')

ticks, _ = plt.yticks()
labels = top_20_ham_trigrams.reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1] + ' ' + t[2])
_ = plt.yticks(ticks, labels)

In [ ]:
data = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_ham_trigrams.to_dict().items()}
img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
plt.figure(figsize=(8,4))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
top_20_spam_trigrams = (pd.Series(nltk.ngrams(spam_words, 3))
                      .value_counts()
                      .head(20))

top_20_spam_trigrams

In [ ]:
top_20_spam_trigrams.plot.barh(color='pink', width=.9, figsize=(10,6))

plt.title('20 Most Frequently Occuring Spam Trigrams')
plt.ylabel('Trigram')
plt.xlabel('Number of Occurances')

ticks, _ = plt.yticks()
labels = top_20_spam_trigrams.reset_index()['index'].apply(lambda t: t[0] + ' ' + t[1] + ' ' + t[2])
_ = plt.yticks(ticks, labels)

In [ ]:
data = {k[0] + ' ' + k[1] + ' ' + k[2]: v for k, v in top_20_spam_trigrams.to_dict().items()}
img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(data)
plt.figure(figsize=(8,4))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
codeup_df = acquire.get_codeup_blogs(cached=True)

In [ ]:
codeup_df

In [ ]:
acquire.prep_text(codeup_df, 'original')

In [ ]:
news_df = get_inshorts_articles()

In [ ]:
news_df

In [ ]:
prep_text(news_df, 'original', extra_words=["'", ','])

In [ ]:
news_df.original[3]

## Create a separate dataframe for each category.

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
list_of_dfs = []
for cat in news_df.category.unique():
    list_of_dfs.append(news_df[news_df.category == cat])

In [ ]:
news_df.clean[0:21]

In [ ]:
news_df.lemmatized[0:4]

In [ ]:
news_df.stemmed[0:4]

# Takeaways
* There is not much difference between the lemmatized and clean versions. The stemmed version is difficult for me to understand. 
* I will work on the clean version of text for exploration.

In [ ]:
news_df.category.unique()

In [ ]:
# Create a corpus of words for each category.
science_words = ' '.join(news_df[news_df.category == 'science'].clean)
business_words = ' '.join(news_df[news_df.category == 'business'].clean)
sports_words = ' '.join(news_df[news_df.category == 'sports'].clean)
technology_words = ' '.join(news_df[news_df.category == 'technology'].clean)
entertainment_words = ' '.join(news_df[news_df.category == 'entertainment'].clean)
all_words = ' '.join(news_df.clean)

In [ ]:
len(science_words), len(business_words), len(sports_words), len(technology_words), len(entertainment_words), len(all_words)

In [ ]:
science_words

### Represent text as word frequencies

In [ ]:
science_freq = pd.Series(science_words.split()).value_counts()
business_freq = pd.Series(business_words.split()).value_counts()
sports_freq = pd.Series(sports_words.split()).value_counts()
technology_freq = pd.Series(technology_words.split()).value_counts()
entertainment_freq = pd.Series(entertainment_words.split()).value_counts()
all_freq = pd.Series(all_words.split()).value_counts()

In [ ]:
word_counts = pd.concat([science_freq, business_freq, sports_freq, technology_freq, entertainment_freq, all_freq], axis=1).fillna(0).astype(int)
word_counts.columns = ['science','business','sports','technology','entertainment','all']

In [ ]:
word_counts

In [ ]:
word_counts['all'].sort_values(ascending=False)

### Visualization
Visualize the top 5 words and determine which category uses those words the most.

In [ ]:
plt.rc('font', size=18)
word_counts.sort_values('all', ascending=False).head(5)[['science', 'technology', 'business', 'sports', 'entertainment']].plot.barh()
plt.title('Top 10 Words for Science and Technology')

# Takeaways
**Out of all words in all categories:**
* 'said' is the most common word.
* Sports news includes 'India' the most.
* Business mentions 'crore' the most.
* The word 'billion' is used the most in technology articles.
* Sports and Entertainment use the word 'added' the most.

## Visualize the top 10 words for each category.

In [ ]:
n = 1
for category in list(word_counts.columns):
    plt.rc('font', size=8)
    plt.figure(figsize=(8,10))
    plt.subplot(3,2,n)
    word_counts[category].sort_values(ascending=False).head(10).plot.barh()
    plt.title(f'Top 10 Words for {category}')
    n += 1

# Takeaways
* 'said' and 'added'(another form of said) are the most commonly used words in news.
* The news is a lot of 'He said/She said/They said"
* Actor is mentioned more than actress in entertainment news.
* Entertainment likes to mention films and Instagram.
* Meta and Facebook dominated technology news.
* Sports news is very nationalistic.
* NASA and space are mentioned a lot in science news.

In [ ]:
stopwords.words('english')

* Find the top 10 longest words for each category.
* Find the average word length for each category.
* Find words that are unique to each category.

I want to create a function that will intake any text and analyze the shit out of it. I want a total character count, character count for the longest word, longest lenth sentence(and that printed), average wordlength per sentence, shortest word(& character count), longest word(& character count), 

In [ ]:
def analyze_text(string):
    # Get length of total characters in all cleaned science articles.
    total_characters = len(string)
    print(f'Total amount of characters: {total_characters}')
    
    # Get wordcount of all words in cleaned science articles.
    total_words = len(string.split())
    print(f'Total amount of words: {total_words}')
    
    # Get list of unique words and a count in cleaned science articles.
    unique_words = pd.DataFrame(string.split())[0].unique()
    print('Total amount of unique words: ',len(unique_words)
          
    # Get length of every unique word and plot a histogram of how many times each length of word appears.
    sns.histplot([len(word) for word in unique_words], binwidth=1)
    plt.xlabel('character_count')
    plt.title('Number of Characters in Each Word')

In [ ]:
# Get length of total characters in all cleaned science articles.
len(science_words)

In [ ]:
# Get wordcount of all words in cleaned science articles.
len(science_words.split())

In [ ]:
# Get list of unique words and a count in cleaned science articles.
unique_words = pd.DataFrame(science_words.split())[0].unique()

len(unique_words)

In [ ]:
# Get length of every unique word and plot a histogram of how many times each length of word appears.
sns.histplot([len(word) for word in unique_words], binwidth=1)
plt.xlabel('character_count')
plt.title('Number of Characters in Each Word')

# Takeaways
* The distribution of word length is slightly right skewed.

In [ ]:
# Get average word length of all words in cleaned science articles.
pd.Series([len(word) for word in unique_words]).mean()